In [110]:
import numpy as np
import scipy as sp
import random
import networkx as nx
import matplotlib as plt
import sys
sys.path.append('..')
sys.path.append('../..')
import os
import argparse
import utils
import networkx as nx
import numpy as np
from student_utils_sp18 import *
from utils import *
from SetCoverV3 import *
from math import sqrt
from math import factorial
from itertools import combinations, permutations

import warnings
warnings.filterwarnings('ignore')




In [111]:
class City:
    def __init__(self, worldmap=None, id=None):
        self.worldmap = worldmap
        self.id = id

    def distanceTo(self, city):
        return self.worldmap[self.id][city.id]

    def __repr__(self):
        return str(self.id)


class TourManager:
    def __init__(self, IS = False):
        self.IS = IS
        self.destinationCities = []

    def addCity(self, city):
        self.destinationCities.append(city)

    def getCity(self, index):
        return self.destinationCities[index]

    def numberOfCities(self):
        return len(self.destinationCities)


class Tour:
    global adjacency_matrix
    global start
    def __init__(self,worldmap, tourmanager, tour=None):
        self.tourmanager = tourmanager
        self.worldmap = worldmap
        self.tour = []
        self.fitness = 0.0
        self.distance = 0
        if tour is not None:
            self.tour = tour
        else:
            for i in range(0, self.tourmanager.numberOfCities()):
                self.tour.append(None)

    def __len__(self):
        return len(self.tour)

    def __getitem__(self, index):
        return self.tour[index]

    def __setitem__(self, key, value):
        self.tour[key] = value

    def __repr__(self):
        geneString = "|"
        for i in range(0, self.tourSize()):
            geneString += str(self.getCity(i)) + "|"
        return geneString

    def generateIndividual(self):
        for cityIndex in range(0, self.tourmanager.numberOfCities()):
            self.setCity(cityIndex, self.tourmanager.getCity(cityIndex))
        random.shuffle(self.tour)

    def getCity(self, tourPosition):
        return self.tour[tourPosition]

    def setCity(self, tourPosition, city):
        self.tour[tourPosition] = city
        self.fitness = 0.0
        self.distance = 0

    def totalTime(self):
        totalTime = self.getDistance()    
        for i in range(self.tourSize()):
            totalTime += adjacency_matrix[self.getCity(i).id][self.getCity(i).id]
        if self.tourmanager.IS:
            totalTime += adjacency_matrix[start][start]

        return totalTime    

    def getFitness(self):
        if self.fitness == 0:
            self.fitness = self.totalTime()
        return self.fitness

    def getDistance(self):
        if self.distance == 0:
            tourDistance = 0
            for cityIndex in range(self.tourSize()-1):
                fromCity = self.getCity(cityIndex)
                destinationCity = self.getCity(cityIndex+1)
                tourDistance += fromCity.distanceTo(destinationCity)
            tourDistance += self.worldmap[start][self.tour[self.tourSize()-1].id]
            tourDistance += self.worldmap[start][self.tour[0].id]
            self.distance = tourDistance


        return self.distance

    def tourSize(self):
        return len(self.tour)


    def containsCity(self, city):
        return city in self.tour


class Population:
    def __init__(self, tourmanager, populationSize, initialise):
        self.tours = []
        for i in range(0, populationSize):
            self.tours.append(None)

        if initialise:
            for i in range(0, populationSize):
                newTour = Tour(distmatrix,tourmanager)
                newTour.generateIndividual()
                self.saveTour(i, newTour)

    def __setitem__(self, key, value):
        self.tours[key] = value

    def __getitem__(self, index):
        return self.tours[index]

    def saveTour(self, index, tour):
         self.tours[index] = tour

    def getTour(self, index):
        return self.tours[index]

    def getFittest(self):
        fittest = self.tours[0]
        for i in range(0, self.populationSize()):
            if fittest.getFitness() > self.getTour(i).getFitness():
                fittest = self.getTour(i)
        return fittest

    def populationSize(self):
        return len(self.tours)




class GA:
    def __init__(self, tourmanager):
        self.tourmanager = tourmanager
        self.mutationRate = 0.015
        self.tournamentSize = 5
        self.elitism = True

    def evolvePopulation(self, pop):
        newPopulation = Population(self.tourmanager, pop.populationSize(), False)
        elitismOffset = 0
        if self.elitism:
            newPopulation.saveTour(0, pop.getFittest())
            elitismOffset = 1

        for i in range(elitismOffset, newPopulation.populationSize()):
            parent1 = self.tournamentSelection(pop)
            parent2 = self.tournamentSelection(pop)
            child = self.crossover(parent1, parent2)
            newPopulation.saveTour(i, child)

        for i in range(elitismOffset, newPopulation.populationSize()):
            self.mutate(newPopulation.getTour(i))

        return newPopulation

    def crossover(self, parent1, parent2):
        child = Tour(distmatrix,self.tourmanager)

        startPos = int(random.random() * parent1.tourSize())
        endPos = int(random.random() * parent1.tourSize())

        for i in range(0, child.tourSize()):
            if startPos < endPos and i > startPos and i < endPos:
                child.setCity(i, parent1.getCity(i))
            elif startPos > endPos:
                if not (i < startPos and i > endPos):
                    child.setCity(i, parent1.getCity(i))

        for i in range(0, parent2.tourSize()):
            if not child.containsCity(parent2.getCity(i)):
                for ii in range(0, child.tourSize()):
                    if child.getCity(ii) == None:
                        child.setCity(ii, parent2.getCity(i))
                        break

        if start in child.tour:
            rec = child.tour[0]
            child.tour.remove(start)
            child.tour[0] = start
            child.tour.append(rec)

        return child

    def mutate(self, tour):
        for tourPos1 in range(0, tour.tourSize()):
            if random.random() < self.mutationRate:
                tourPos2 = int(tour.tourSize() * random.random())

                city1 = tour.getCity(tourPos1)
                city2 = tour.getCity(tourPos2)

                tour.setCity(tourPos2, city1)
                tour.setCity(tourPos1, city2)
        if start in tour:
            rec = tour[0]
            tour.remove(start)
            tour[0] = start
            tour.append(rec)

    def tournamentSelection(self, pop):
        tournament = Population(self.tourmanager, self.tournamentSize, False)
        for i in range(0, self.tournamentSize):
            randomId = int(random.random() * pop.populationSize())
            tournament.saveTour(i, pop.getTour(randomId))
        fittest = tournament.getFittest()
        return fittest



def toBinary(x):
    n = len(x)
    ret = [[0] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            if x[i][j] != 'x':
                   ret[i][j] = 1
    return ret

def toReal(x):
    INF  = 9999999999999999999999999999999999999999999
    n = len(x) # Get size of matrix
    ret = [[INF] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            if x[i][j] != 'x':
                   ret[i][j] = x[i][j]
    for i in range(n):
        ret[i][i] = 0
    return ret


# Solves all pair shortest path via Floyd Warshall Algrorithm
def floydWarshall(graph):
    dist = list(map(lambda i : list(map(lambda j : j , i)) , graph))
    paths = [[str(list_of_kingdom_names[i])]*n for i in range(n)]
    for k in range(V):
        # pick all vertices as source one by one
        for i in range(V):
            # Pick all vertices as destination for the
            # above picked source
            for j in range(V):
                # If vertex k is on the shortest path from
                # i to j, then update the value of dist[i][j]
                if (dist[i][j] > dist[i][k]+ dist[k][j]):
                    dist[i][j] = dist[i][k]+ dist[k][j]
                    paths[i][j] = paths[i][k]+" "+paths[k][j]

    return dist,paths

In [125]:
for blabla in range(725, 800):
    if blabla not in [4, 102, 103, 104, 210, 211, 212, 711, 712, 713, 705, 706, 707, 696, 697, 698, 521, 375, 376, 377, 310, 249, 250]:
        print(blabla)
        inp = "inputs/"+str(blabla)+".in"
        data = read_file(inp)
        number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = data_parser(data)
        n = number_of_kingdoms
        INF  = 9999999999999999999999999999999999999999999
        G = adjacency_matrix_to_graph(adjacency_matrix)

        binmatrix = toBinary(adjacency_matrix)
        realmatrix = toReal(adjacency_matrix)
        V = n
        start = list_of_kingdom_names.index(starting_kingdom)


        output1 = "output/"+str(blabla)+".out"
        f = open(output1, 'r')
        x = f.readlines()
        f.close()
        p = x[0]
        ori_path = []
        rec = ''
        for i in p:
            if i != ' ':
                rec += i
            else:
                ori_path.append(list_of_kingdom_names.index(rec))
                rec = ''
        if ori_path == []:
            ori_path.append(list_of_kingdom_names.index(rec.rstrip()))
        print(ori_path)
        f = open(output1, 'r')
        x = f.readlines()
        f.close()
        p = x[1]
        to_con = []
        rec = ''
        for i in p:
            if i != ' ':
                rec += i
            else:
                to_con.append(list_of_kingdom_names.index(rec))
                rec = ''
        if rec != '\n':
            to_con.append(list_of_kingdom_names.index(rec.rstrip()))
        distmatrix, paths= floydWarshall(realmatrix)
        print(to_con)

        if __name__ == '__main__':
            a = 1
            b = 0
            c = 0
            d = 1
            ori = 0
            for i in to_con:
                ori += adjacency_matrix[i][i]
            for i in range(len(ori_path)-1):
                ori += adjacency_matrix[ori_path[i]][ori_path[i+1]]
            ori += adjacency_matrix[ori_path[len(ori_path)-1]][start]

            result1 = to_con
            length1 = len(result1)
            if length1 <= 10:
                possibilities = list(permutations(result1, length1))
                first_possibility = list(possibilities[0])
                shortest = distmatrix[start][first_possibility[0]]
                index = 0
                for i in range(length1-1):
                    shortest += distmatrix[first_possibility[i]][first_possibility[i+1]]
                    shortest += adjacency_matrix[first_possibility[i]][first_possibility[i]]
                shortest += distmatrix[first_possibility[length1-1]][start]
                shortest += adjacency_matrix[first_possibility[length1-1]][first_possibility[length1-1]]
                for i in range(1, factorial(length1)):
                    chosen = list(possibilities[i])
                    new_distance = distmatrix[start][chosen[0]]
                    for j in range(length1-1):
                        new_distance += distmatrix[chosen[j]][chosen[j+1]]
                        new_distance += adjacency_matrix[chosen[j]][chosen[j]]
                    new_distance += distmatrix[chosen[length1-1]][start]
                    new_distance += adjacency_matrix[chosen[length1-1]][chosen[length1-1]]
                    if new_distance < shortest:
                        shortest = new_distance
                        index = i
                result1 = list(possibilities[index])

                if start in result1:
                    tourmanager1 = TourManager(True)
                    count = 0
                    for i in result1:
                        if i != start:
                            city = City(distmatrix, i)
                            tourmanager1.addCity(city)
                else:
                    tourmanager1 = TourManager()
                    for i in result1:
                        city = City(distmatrix, i)
                        tourmanager1.addCity(city)
                soln = Tour(distmatrix, tourmanager1)

                for cityIndex in range(0, tourmanager1.numberOfCities()):
                    soln.setCity(cityIndex, soln.tourmanager.getCity(cityIndex))
                cost = soln.totalTime()
            else:
                if start in result1:
                    tourmanager1 = TourManager(True)
                    for i in result1:
                        if i != start:
                            city = City(distmatrix, i)
                            tourmanager1.addCity(city)
                else:
                    tourmanager1 = TourManager()
                    for i in result1:
                        city = City(distmatrix, i)
                        tourmanager1.addCity(city)
                pop1 = Population(tourmanager1, 150, True)
                ga1 = GA(tourmanager1)
                for i in range(0, 1000):
                    pop1 = ga1.evolvePopulation(pop1)
                soln = pop1.getFittest()
                cost = soln.totalTime()

            orig_stdout = sys.stdout
            realpath=[]
            realpath += [paths[start][soln.tour[0].id]]
            for i in range(len(soln.tour)-1):
                realpath += [paths[soln.tour[i].id][soln.tour[i+1].id]]
            realpath += [paths[soln.tour[len(soln.tour)-1].id][start]]
            if soln.tourmanager.IS:
                record = []
                record.append(City(distmatrix, start))
                for city in soln.tour:
                    record.append(city)
                soln.tour = record
            if soln.tour[len(soln.tour)-1].id != start:
                realpath += [list_of_kingdom_names[start]]
            out = "newoutputs/"+str(blabla)+".out"
            f = open(out, 'w')
            sys.stdout = f

            print(ori)
            print(cost)

            print(' '.join(['{:1}'.format(item) for item in realpath]))
            print(' '.join(['{:1}'.format(list_of_kingdom_names[int(item.id)]) for item in soln]))
            sys.stdout = orig_stdout
            f.close()

725
[49, 8, 12, 8, 7, 24]
[12, 7, 24]
726
[0, 72, 82, 45, 82, 22, 78, 11, 52, 4, 88, 81, 88, 4, 17, 54, 82, 96, 69, 68, 69, 83, 13, 2, 93, 64, 71, 86, 50]
[45, 78, 52, 81, 88, 17, 96, 68, 83, 13, 93, 64, 71, 86]
727
[34, 138, 127, 138, 125, 77, 84, 75, 91, 33, 93, 100, 52, 113, 52, 21, 47, 197, 63, 40, 174, 160, 174, 54, 99, 20, 31, 6, 92, 14, 85, 14, 104, 193, 1, 185, 188, 60, 142, 172, 176, 89, 176, 172, 116, 34, 98, 195, 88, 90, 182, 4, 23, 49, 70, 9, 56, 48, 88, 10, 104, 181, 82, 184, 79, 184, 65, 159, 68, 165, 123, 26, 156, 177, 156, 8, 176, 172, 142]
[34, 127, 125, 75, 33, 100, 52, 113, 47, 197, 174, 160, 20, 6, 85, 104, 193, 1, 185, 60, 142, 172, 89, 116, 98, 88, 182, 23, 49, 70, 48, 10, 82, 184, 79, 68, 177]
728
[42, 6, 14, 33, 23, 26, 23, 33, 14, 6, 14, 49, 27, 36, 31, 24, 31, 22, 1]
[26, 33, 6, 36, 24, 22]
729
[28, 41, 35, 19, 68, 87, 96, 74, 50, 66, 60, 72, 60, 6, 80, 13, 90, 44, 38, 49, 34, 75, 89, 55, 30, 37, 40, 94, 78, 85, 26, 85, 61, 46, 14, 8, 70, 24, 97]
[28, 68, 74, 

FileNotFoundError: [Errno 2] No such file or directory: 'inputs/753.in'